In [ ]:
!rm -vr ~/.kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
!chmod 777 ~/.kaggle/kaggle.json

In [ ]:
# from ultralytics import YOLO
# from roboflow import Roboflow
# rf = Roboflow(api_key="rEI3w1C5xkPJcmefNDI3")
# project = rf.workspace("sign-arabic-language").project("final-project-kpbmm")
# version = project.version(5)
# dataset = version.download("yolov8")

In [ ]:
!pip install ultralytics


In [ ]:
api_token = {"username":"azzaali","key":"77d4f57492eab11f39f0a5babfc0ed29"}
import json
with open('/root/.kaggle/kaggle.json', 'w') as file:
   json.dump(api_token, file)
!chmod 600 ~/.kaggle/kaggle.json
# !/bin/bash
!kaggle datasets download -d ammarsayedtaha/arabic-sign-language-dataset-2022
# %%capture
!unzip /content/arabic-sign-language-dataset-2022.zip

## Libraries

In [ ]:
from ultralytics import YOLO
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random


## Visualization

In [ ]:
# def read_labels(label_path):
#     with open(label_path, 'r') as file:
#         lines = file.readlines()
#         labels = [line.strip().split() for line in lines]
#     return labels

# def draw_boxes(image, labels):
#     for label in labels:
#         class_id = int(label[0])
#         x, y, w, h = map(float, label[1:])
#         image_height, image_width, _ = image.shape
#         x1 = int((x - w / 2) * image_width)
#         y1 = int((y - h / 2) * image_height)
#         x2 = int((x + w / 2) * image_width)
#         y2 = int((y + h / 2) * image_height)
#         color = (0, 255, 0)  # Green
#         cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
#     return image

# data_dir = '/content/datasets/train'

# image_files = [file for file in os.listdir(os.path.join(data_dir, 'images')) if file.endswith('.jpg')]

# random.shuffle(image_files)

# fig, axes = plt.subplots(3, 3, figsize=(12, 12))
# for ax, image_file in zip(axes.ravel(), image_files[:9]):
#     image_path = os.path.join(data_dir, 'images', image_file)
#     label_path = os.path.join(data_dir, 'labels', os.path.splitext(image_file)[0] + '.txt')

#     # Read image
#     image = cv2.imread(image_path)
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Read labels
#     labels = read_labels(label_path)

#     # Draw bounding boxes on the image
#     image_with_boxes = draw_boxes(image_rgb, labels)

#     # Display image with bounding boxes and set the labels as titles
#     ax.imshow(image_with_boxes)
#     ax.set_title(str(labels[0][0]))
#     ax.axis('off')

# plt.tight_layout()
# plt.show()

## Model

In [ ]:
model = YOLO('yolov8l.yaml')
model = YOLO('yolov8l.pt')
model = YOLO('yolov8l.yaml').load('yolov8l.pt')

In [ ]:
history = model.train(data='/content/datasets/sign.yaml', epochs=100, imgsz=640,
                    patience = 100, batch = 64,
                    project ="ASL", optimizer = 'Adam', momentum = 0.9,
                    cos_lr=True ,seed = 42, plots = True , close_mosaic = 0, lr0 = 0.001)

## Loading Model (Previously Trained Model Saved)

In [ ]:
# # loading Model
# trained_model = YOLO('ASL.pt')

## Predictions

In [ ]:
test_images_dir = '/content/datasets/valid/images'
test_images = [os.path.join(test_images_dir, image) for image in os.listdir(test_images_dir)]

test_samples = np.random.choice(test_images, 10, replace=False)

print(test_samples)

In [ ]:
results = model([test_samples[0], test_samples[1], test_samples[2], test_samples[3], test_samples[4], test_samples[5], test_samples[6], test_samples[7], test_samples[8], test_samples[9]])

results_dir = "results_tries"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

i = 0
for result in results:
    boxes = result.boxes
    masks = result.masks
    keypoints = result.keypoints
    probs = result.probs

    # Save image
    filename = os.path.join(results_dir, f"result_{i}.jpg")
    result.save(filename=filename)
    # result.show() # Uncomment to display image
    i+=1


In [ ]:
directory = "results_tries"

images = []
for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
        img = plt.imread(os.path.join(directory, filename))
        images.append(img)

fig, axs = plt.subplots(2, 5, figsize=(50, 25))
for i in range(10):
    axs[i//5, i%5].imshow(images[i])
    axs[i//5, i%5].axis('off')

plt.show()

## Inference (Use this code when you have your saved trained model + a webcam to capture the frames)

In [ ]:
# import cv2
# from ultralytics import YOLO

# # Load the model
# model = YOLO('ASL.pt') # Your saved model's name

# # Open the camera
# cap = cv2.VideoCapture(0)

# # Setting width and height of the video
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)


# while cap.isOpened():

#     success, frame = cap.read()

#     if success:

#         results = model.track(frame, persist=True)


#         annotated_frame = results[0].plot()

#         cv2.imshow("ASL Tracking", annotated_frame)

#         # Break the loop if 'q' is pressed
#         if cv2.waitKey(1) & 0xFF == ord("q"):
#             break
#     else:
#         # Break the loop if
#         break

# cap.release()
# cv2.destroyAllWindows()
